In [1]:
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import time

In [3]:
def yearly_history_by_week(ticker_name):
    history_df = yf.download(tickers=ticker_name, period='1y').reset_index()[['Date','Close']]

    history_df.loc[:,'year'] = [date.isocalendar()[0] for date in history_df['Date']]
    history_df.loc[:,'week'] = [date.isocalendar()[1] for date in history_df['Date']]

    weekly_history_df = history_df[['year','week','Close']].groupby(['year','week']).mean().reset_index()

    weekly_history_df.loc[:,'close'] = weekly_history_df['Close'].round(1)
    weekly_history_df = weekly_history_df.drop(columns = ['Close']).rename(columns = {'close': ticker_name.lower()})
    
    return weekly_history_df

In [12]:
bsx = yf.Ticker("BSX")

In [40]:
bsx.quarterly_balance_sheet

,2021-09-30,2021-06-30,2021-03-31,2020-12-31
Intangible Assets,6.365000e+09,5.954000e+09,6.182000e+09,6.135000e+09
Capital Surplus,1.993000e+10,1.981700e+10,1.975000e+10,1.973200e+10
Total Liab,1.541500e+10,1.522600e+10,1.518100e+10,1.545100e+10
Total Stockholder Equity,1.646200e+10,1.594200e+10,1.571900e+10,1.532600e+10
Other Current Liab,1.279000e+09,1.355000e+09,1.143000e+09,1.419000e+09
Total Assets,3.187700e+10,3.116800e+10,3.090000e+10,3.077700e+10
Common Stock,1.700000e+07,1.700000e+07,1.700000e+07,1.700000e+07
Other Current Assets,1.010000e+09,9.780000e+08,1.047000e+09,2.078000e+09
Retained Earnings,-1.473000e+09,-1.878000e+09,-2.050000e+09,-2.378000e+09
Other Liab,2.192000e+09,1.947000e+09,2.353000e+09,2.239000e+09


In [17]:
bsx.quarterly_cashflow

,2021-09-30,2021-06-30,2021-03-31,2020-12-31
Investments,1.860000e+08,6.700000e+07,25000000.0,-16000000.0
Change To Liabilities,8.800000e+07,1.970000e+08,-10000000.0,88000000.0
Total Cashflows From Investing Activities,-1.207000e+09,-1.207000e+09,71000000.0,-147000000.0
Total Cash From Financing Activities,6.000000e+06,2.000000e+06,-95000000.0,-819000000.0
Change To Operating Activities,-9.800000e+07,6.500000e+07,-112000000.0,-316000000.0
Issuance Of Stock,6.400000e+07,2.000000e+07,18000000.0,4000000.0
Net Income,4.190000e+08,1.860000e+08,341000000.0,210000000.0
Change In Cash,-7.390000e+08,6.450000e+08,258000000.0,-287000000.0
Effect Of Exchange Rate,-3.000000e+06,1.000000e+06,-3000000.0,6000000.0
Total Cash From Operating Activities,4.650000e+08,6.430000e+08,284000000.0,673000000.0


In [39]:
bsx.earnings

,Revenue,Earnings
Year,,
2017,9048000000,104000000
2018,9823000000,1671000000
2019,10735000000,4700000000
2020,9913000000,-82000000


In [36]:
bsx_earnings = bsx.quarterly_earnings
bsx_earnings

,Revenue,Earnings
Quarter,,
4Q2020,2709000000,210000000
1Q2021,2752000000,341000000
2Q2021,3077000000,186000000
3Q2021,2932000000,419000000


In [37]:
bsx_earnings.loc[:,'result'] = bsx_earnings['Revenue'] - bsx_earnings['Earnings']

In [38]:
bsx_earnings

,Revenue,Earnings,result
Quarter,,,
4Q2020,2709000000,210000000,2499000000
1Q2021,2752000000,341000000,2411000000
2Q2021,3077000000,186000000,2891000000
3Q2021,2932000000,419000000,2513000000


In [24]:
baba_recommendations = yf.Ticker("BABA").recommendations

In [6]:
my_portfolio_tickers = ['ABBV','BABA','JD','BSX']

history_df = yearly_history_by_week('EXC')

for company in my_portfolio_tickers:
    history_df = history_df.merge(yearly_history_by_week(company), left_on = ['year','week'], right_on = ['year','week'])
    print(company, end = ' ')
    time.sleep(3)

history_df.head(4)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
BSX 

,year,week,exc,abbv,baba,jd,bsx
0,2020,47,42.1,100.3,265.3,86.9,34.0
1,2020,48,42.1,103.8,276.1,88.8,33.6
2,2020,49,41.2,105.1,264.6,85.3,33.8
3,2020,50,41.2,107.2,264.7,82.3,34.0
